# Đồ án cuối kì Lập trình cho khoa học dữ liệu


## Thành viên nhóm


| |Họ và tên|MSSV|
|:--:|:--|:--:|
|1|Võ Thành Nam|19120301|
|2|Phạm Lưu Mỹ Phúc|19120331|

## Môi trường code

In [ ]:
import sys
sys.executable

## Tổng quan về dữ liệu sẽ được dùng để phân tích

Bộ dữ liệu được sử dụng là file `DataScientist.csv`. Đây là dữ liệu nằm trong một bộ dữ liệu gồm nhiều file khác về thông tin tuyển dụng của những ngành nghề liên quan đến Data. 
- Bộ dữ liệu được tác giả *picklesueat* lấy từ trang web https://www.glassdoor.com/ (một trang web tìm kiếm việc làm), sử dụng công cụ `selenium`. 
- Bộ dữ liệu đã được tác giả public tại [Github](https://github.com/picklesueat/data_jobs_data) và được reup tại [Kaggle](https://www.kaggle.com/andrewmvd/data-scientist-jobs).
- Về license của dữ liệu, tác giả không đề cập cụ thể mà cho phép mọi người có thể tải xuống tùy ý.
- File dữ liệu `DataScientist.csv` là file dữ liệu về thông tin tuyển dụng của nghề Data Scientist bao gồm tên vị trí tuyển dụng, mức lương, tên công ty,...

## Import các thư viện cần thiết

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib as plt

## Khám phá dữ liệu

### Đọc dữ liệu vào từ file csv vào dataframe

In [ ]:
ds_df = pd.read_csv('DataScientist.csv')
ds_df.head()

Đầu tiên, ta thấy dữ liệu có đến 2 cột index, ta sẽ không sử dụng 2 cột này, do đó sẽ loại bỏ 2 cột này đi.

In [ ]:
ds_df.drop(columns = ['Unnamed: 0','index'],inplace = True)
ds_df.head()

### Số lượng dòng và cột của dữ liệu

In [ ]:
n_rows = ds_df.shape[0]
n_cols = ds_df.shape[1]
print(f'Number of rows: {n_rows}')
print(f'Number of columns: {n_cols}')

### Xét xem các dữ liệu có trùng nhau hay không

In [ ]:
print(f'Number of duplicated rows: {sum(ds_df.duplicated())}')

### Ý nghĩa của các cột trong dataframe
(Dựa theo mô tả tại trang web [Kaggle](https://www.kaggle.com/andrewmvd/data-scientist-jobs))

- Job Title: tên vị trí tuyển dụng.
- Salary Estimate: mức lương dự kiến.
- Job Description: mô tả công việc.
- Rating: điểm đánh giá.
- Company Name: tên công ty.
- Location: địa điểm làm việc.
- Headquarters: trụ sở công ty.
- Size: kích thước công ty (số lượng nhân viên).
- Founded: năm thành lập công ty.
- Type of ownership: loại hình sở hữu của công ty.
- Industry: phân loại ngành của công ty.
- Sector: lĩnh vực của công ty.
- Revenue: doanh thu của công ty (theo năm).
- Competitors: công ty đối thủ.
- Easy Apply: công ty có dễ apply hay không.

### Sơ lược về dữ liệu trong các cột

In [ ]:
ds_df.info()

### Nhận xét chung

- Như ta có thể thấy, dữ liệu không có bất kì dòng nào thiếu hay trùng lắp. Tuy nhiên, khi quan sát bảng dữ liệu ở phía trên, có thế thấy tác giả đã thay thế các dữ liệu (có thể là bị thiếu) bằng giá trị -1.
- Những cột đáng lẽ cần phải ở kiểu dữ liệu là số như `Salary Estimate`, `Size` thì lại có kiểu dữ liệu là object, ta cần phải xử lí những cột này.
- Cột `Company Name` có vẻ bị lỗi khi thu thập dữ liệu (tên công ty gắn liền với rating).
- Cột `Easy Apply` chỉ nên có giá trị `True` hoặc `False`.
- ...

Ta không quan sát được toàn bộ dữ liệu, do đó, ta sẽ phân tích và tiền xử lí cho từng cột trong phần tiền xử lí dữ liệu.

## Tiền xử lí dữ liệu

In [ ]:
ds_df.head()

### Các giá trị của Job title liệu đã đúng hay chưa?

Quan sát các dòng đầu tiên, ta thấy `Job Title` không chỉ bao gồm một vị trí mà có thế có nhiều vị trí, ta sẽ xem xét các giá trị trong cột này.

In [ ]:
ds_df['Job Title'].nunique()

Số lượng các `Job Title` khác nhau là rất lớn, do đó ta sẽ xem qua 20 giá trị xuất hiện nhiều nhất và 20 giá trị xuất hiện ít nhất.

In [ ]:
job_title_first_20 = ds_df['Job Title'].value_counts()[:20].reset_index()
job_title_last_20 = ds_df['Job Title'].value_counts()[-20:].reset_index()
pd.concat([job_title_first_20,job_title_last_20],axis=1).rename(columns = {'Job Title':'count','index':'Job Title'})

Ta có nhận xét: mỗi công ty khi đăng tin tuyển dụng sẽ có một cách thông tin về **Job title** riêng (có thể là tuyển dụng cho nhiều vị trí cùng lúc, ghi tên vị trí cùng với thời gian làm việc,...) nên số lượng giá trị là rất lớn. Tuy nhiên cũng có nhiều vị trí trùng nhau nhưng lại có cách viết khác nhau (Sr., Senior), ta sẽ xử lí những giá trị này.

In [ ]:
ds_df['Job Title'] = ds_df['Job Title'].str.replace('Sr.','Senior',regex=False)
ds_df['Job Title'] = ds_df['Job Title'].str.replace('Jr.','Junior',regex=False)

### Loại bỏ rating trong Company name

In [ ]:
ds_df['Company Name'] = ds_df['Company Name'].str.split('\n').str[0]

### Tách tên thành phố trong cột Headquarters và xử lí missing value

In [ ]:
ds_df['Headquarters'].replace('-1',np.nan,inplace = True)

In [ ]:
headquarters_splited = ds_df['Headquarters'].str.split(',',expand=True)
headquarters_splited

Một số tên công ty có định dạng thông tin `Headquarters` khác với các công ty còn lại, ta sẽ xem xét và giải quyết những giá trị đặc biệt này.

In [ ]:
headquarters_splited[~headquarters_splited[2].isnull()]

Như vậy, các định dạng khác thường là do một số công ty có ghi thêm quốc gia, ta sẽ loại bỏ các giá trị này.

In [ ]:
headquarters_splited[1] = headquarters_splited[1].str.replace('\(\w*\)','',regex=True) 
headquarters_splited.drop(columns=[2],inplace=True)

In [ ]:
ds_df[['Headquarters_City','Headquarters_State/Country']] = headquarters_splited
ds_df.drop(columns=['Headquarters'],inplace = True)

In [ ]:
ds_df

### Tiền xử lí các cột Founded, Industry, Revenue

In [ ]:
fir_df = ds_df[['Founded','Industry','Revenue']]
fir_df

## Đưa ra các câu hỏi cần trả lời

Câu 1

Câu 2

Câu 3

### Câu hỏi 1:

### Câu hỏi 2:

### Câu hỏi 3:

## Tổng kết

## Tài liệu tham khảo